In [ ]:
import requests
import xml.etree.ElementTree as ET
import numpy as np
from equilibrator_api import ComponentContribution

cc = ComponentContribution()
s = requests.Session() # create 

In [ ]:
# from Kegg id to inchi_key

kegg_id = 'C00031'

for i in cc.search_compound(kegg_id).identifiers:
    if i.registry.namespace == 'inchi_key':
        print(i.accession)

In [ ]:
r = s.get('https://api.datanator.info/metabolites/concentration_only/?inchi_key='+inchi_key)
np.mean(list(map(float,r.text[r.text.find('concentration": [')+17:r.text.find('],\n      "concentration_units')].split('"')[1::2])))